In [ ]:
import sys, os
sys.path.append(os.path.abspath('package'))
from package import KnowledgeGraphBuilder
import pandas as pd

# 1. Build knowledge graph

Initialize the `KnowledgeGraphBuilder` object. A github token must be given as a parameter.

In [2]:
with open ('./_/git_token.txt', 'r') as f:
    git_token = f.read().strip()

with open ('./_/hf_token.txt', 'r') as f:
    hf_token = f.read().strip()

kgb = KnowledgeGraphBuilder(git_token, hf_token)

Build the knowledge graph with the `KnowledgeGraphBuilder.build_knowledge_graph()` method. Parameters:
- *repo_name*: Name of the repository. Must match the format "owner/repo_name", as it is used for github API calls.
- *graph_type* (optional): Type of subgraph to build from the functions. Can be "CFG" (Control Flow Graph) or "AST" (Abstract Syntax Tree). Default is "CFG".
- *num_of_PRs* (optional): Number of pull requests to retrieve in detail. Defaults to 0 (all).
- *create_embedding* (optional): Whether to create embeddings for the nodes. Defaults to False.
- *repo_path_modifier* (optional): Path modifier for the repository for cases when only a subfolder is meant to be parsed.
- *URI* (optional): URI for the Neo4J data saving.
- *user* (optional): Username for the Neo4J data saving.
- *password* (optional): Password for the Neo4J data saving.

In [ ]:
repograph = kgb.build_knowledge_graph(
    repo_name='scikit-learn/scikit-learn',
    num_of_PRs=5,
    num_of_issues=20,
    scrape_comments=False,
    repo_path_modifier='sklearn/manifold',
    semantic_clustering=True,
    create_embedding=False,
)

Repo already exists here: ./repos\scikit-learn
Building CG...
Creating subgraphs for each function...


151it [00:00, 206.00it/s]


Subgraphs created.
Filtering graph nodes...
Graph nodes filtered. Creating hierarchical edges...
Hierarchical graph building successful.


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 112.35it/s]
Device set to use cuda:0


Number of clusters: 49 with silhouette score: 0.18019589030982353
Function nodes clustered.


KeyboardInterrupt: 

In [7]:
repograph.keys()

dict_keys(['function_nodes', 'function_edges', 'subgraph_nodes', 'subgraph_edges', 'subgraph_function_edges', 'function_subgraph_edges', 'import_nodes', 'class_nodes', 'class_function_edges', 'file_nodes', 'file_edges', 'file_function_edges', 'file_class_edges', 'file_import_edges', 'config_nodes', 'file_config_edges', 'import_function_edges', 'pr_nodes', 'pr_function_edges', 'issue_nodes', 'issue_pr_edges', 'artifacts', 'actions', 'cluster_nodes', 'cluster_function_edges', 'functionversion_nodes', 'functionversion_edges', 'functionversion_function_edges', 'developer_nodes', 'developer_function_edges', 'question_nodes', 'question_cluster_edges', 'clusterensemble_nodes', 'clusterensemble_edges'])

# 2. Visualize graph

Create a HTML visualizaiton of the graph with the `visualize_graph` function. NOTE: for large graphs, it is advised to only plot a fraction of the nodes, othervise the visualization might not render properly. Parameters:
- *repograph*: The dictionary containing the created repository graph.
- *show_subgraph_nodes* (optional): Whether to plot the subgraph (CFG or AST) nodes. Defaults to *False*.
- *save_path* (optional): The file path to save the visualization. Defaults to "./graph.html".

In [4]:
kgb.visualize_graph(repograph, show_subgraph_nodes=False)

Graph visualization saved to ./graph.html


# 3. Save the graph

Saving the graph in different formats.

### 3.1 Save it as a dictionary

Saving and loading the resulting graph dictionary as a pickle.

In [11]:
import pickle

with open('graph_TEST.pkl', 'wb') as f:
    pickle.dump(repograph, f)

In [3]:
import pickle

with open('graph_TEST.pkl', 'rb') as f:
    repograph = pickle.load(f)

In [ ]:
repograph.keys()

### 3.2 Saving it to Neo4j database

The result can be saved to a Neo4j database by calling the `store_knowledge_graph_in_neo4j` method. Parameters:
- *URI*: URI for the Neo4J data saving.
- *user*: Username for the Neo4J data saving.
- *password*: Password for the Neo4J data saving.
- *knowledge_graph*: The knowledge graph to save.

If the *URI*, *username* and *password* parameters are provided at the `build_knowledge_graph` method, this function will automatically be called and the graph will be saved to neo4j.

In [ ]:
kgb.store_knowledge_graph_in_neo4j(
    URI="neo4j://127.0.0.1:7687",
    user="neo4j",
    password="password",
    knowledge_graph=repograph
)